In [3]:
import json
import os
from datetime import datetime
from itertools import permutations, combinations, product

def load_common_mpins_6digit():
    common_mpins = set()
    digits = '0123456789'

    # 1. All same digits: '000000', ..., '999999'
    for d in digits:
        common_mpins.add(d * 6)

    # 2. Palindromes: ABCCBA, AABBCC, ABCABC, etc.
    for a in digits:
        for b in digits:
            for c in digits:
                common_mpins.add(f"{a}{b}{c}{c}{b}{a}")  # palindrome
                common_mpins.add(f"{a}{a}{b}{b}{c}{c}")  # block palindrome
                common_mpins.add(f"{a}{b}{c}{a}{b}{c}")  # repeated triplet

    # 3. Repeating patterns: ABABAB, AAABBB, AABBAA, ABCCBA, BAABAA, etc.
    for a, b in product(digits, repeat=2):
        common_mpins.add(f"{a}{b}{a}{b}{a}{b}")
        common_mpins.add(f"{a}{a}{a}{b}{b}{b}")
        common_mpins.add(f"{a}{a}{b}{b}{a}{a}")
        common_mpins.add(f"{a}{a}{b}{b}{b}{b}")

    # 4. Ascending/descending/consecutive runs (no wrap, also all permutations if unique)
    for start in range(0, 5):  # 0-4
        seq = ''.join(str(start + i) for i in range(6))
        common_mpins.add(seq)
        if len(set(seq)) == 6:
            for p in permutations(seq):
                common_mpins.add(''.join(p))
    for start in range(9, 4, -1):  # 9-5
        seq = ''.join(str(start - i) for i in range(6))
        common_mpins.add(seq)
        if len(set(seq)) == 6:
            for p in permutations(seq):
                common_mpins.add(''.join(p))

    # 5. Arithmetic progressions (step 1-4, mod 10)
    for step in range(1, 5):
        for start in range(0, 10):
            seq = [str((start + i * step) % 10) for i in range(6)]
            sseq = ''.join(seq)
            common_mpins.add(sseq)
            if len(set(seq)) == 6:
                for p in permutations(seq):
                    common_mpins.add(''.join(p))

    return common_mpins

def is_valid_mpin(mpin):
    return isinstance(mpin, str) and mpin.isdigit() and len(mpin) == 6

def is_valid_phone_number(phone_number):
    return isinstance(phone_number, str) and phone_number.isdigit() and len(phone_number) == 10

def is_valid_pincode(pincode):
    return isinstance(pincode, str) and pincode.isdigit() and len(pincode) == 6

def is_valid_date(date, is_dob=True):
    if not date or '-' not in date:
        return False
    try:
        day, month, year = map(int, date.split('-'))
        if not (1 <= day <= 31 and 1 <= month <= 12 and 1900 <= year <= 2025):
            return False
        if month in [4, 6, 9, 11] and day > 30:
            return False
        if month == 2:
            is_leap = (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)
            if (is_leap and day > 29) or (not is_leap and day > 28):
                return False
        input_date = datetime(year, month, day)
        current_date = datetime(2025, 6, 30)
        if is_dob and input_date > current_date:
            return False
        if not is_dob and (input_date > current_date or year < 1900):
            return False
        return True
    except (ValueError, TypeError):
        return False

def prompt_valid_date(prompt, is_dob=True):
    while True:
        date = input(prompt).strip()
        if not date:
            return ""
        if is_valid_date(date, is_dob):
            return date
        error_msg = (
            "Invalid date. Use DD-MM-YYYY format (e.g., 02-01-1998). "
            f"{'DOB must not be in the future.' if is_dob else 'Anniversary must be between 1900 and June 30, 2025.'} "
            "Try again or press Enter to skip."
        )
        print(error_msg)

def load_demographics(phone_number):
    if os.path.exists('demographics.json'):
        try:
            with open('demographics.json', 'r') as file:
                data = json.load(file)
            return data.get(phone_number, None)
        except (json.JSONDecodeError, IOError):
            print("Error reading demographics.json. Please enter demographics manually.")
    return None

def get_manual_demographics(phone_number):
    demographics = {"phone_number": phone_number} if is_valid_phone_number(phone_number) else {}
    print("Enter demographics (format: DD-MM-YYYY for dates, press Enter to skip):")
    demographics['dob'] = prompt_valid_date("Your Date of Birth (DD-MM-YYYY): ", is_dob=True)
    demographics['father_dob'] = prompt_valid_date("Father's Date of Birth (DD-MM-YYYY): ", is_dob=True)
    demographics['mother_dob'] = prompt_valid_date("Mother's Date of Birth (DD-MM-YYYY): ", is_dob=True)
    demographics['parents_anniversary'] = prompt_valid_date("Parents' Anniversary (DD-MM-YYYY): ", is_dob=False)
    demographics['spouse_dob'] = prompt_valid_date("Spouse's Date of Birth (DD-MM-YYYY): ", is_dob=True)
    demographics['anniversary'] = prompt_valid_date("Your Anniversary (DD-MM-YYYY): ", is_dob=False)
    children_dobs = []
    for i in range(1, 4):
        child_dob = prompt_valid_date(f"Child {i} Date of Birth (DD-MM-YYYY, press Enter to stop): ", is_dob=True)
        if not child_dob:
            break
        children_dobs.append(child_dob)
    if children_dobs:
        demographics['children_dobs'] = children_dobs
    siblings_dobs = []
    for i in range(1, 4):
        sibling_dob = prompt_valid_date(f"Sibling {i} Date of Birth (DD-MM-YYYY, press Enter to stop): ", is_dob=True)
        if not sibling_dob:
            break
        siblings_dobs.append(sibling_dob)
    if siblings_dobs:
        demographics['siblings_dobs'] = siblings_dobs
    while True:
        pincode = input("Your Pincode (6 digits): ").strip()
        if not pincode or is_valid_pincode(pincode):
            demographics['pincode'] = pincode
            break
        print("Invalid pincode. Must be a 6-digit number. Try again or press Enter to skip.")
    return demographics if demographics else None

def get_date_segments(date):
    """Returns all 2-digit (zero-padded) day, month, 'year', and last 2-digits of year as strings."""
    if not date or not is_valid_date(date):
        return []
    day, month, year = date.split('-')
    year = year.zfill(4)
    return [day.zfill(2), month.zfill(2), year[-2:], year]

def extract_number_mpins(number, num_digits=6):
    """All possible consecutive runs of num_digits from number."""
    mpins = set()
    if number and number.isdigit() and len(number) >= num_digits:
        for i in range(len(number) - num_digits + 1):
            part = number[i:i+num_digits]
            mpins.add(part)
    return mpins

def generate_demographic_mpins(demographics):
    """All relevant MPINs from demographics using 6 digits."""
    all_mpin_set = set()
    date_fields = [
        "dob", "father_dob", "mother_dob", "parents_anniversary",
        "spouse_dob", "anniversary"
    ]
    # Get segments as flat list of all date sub-parts
    segments_by_date = []
    for field in date_fields:
        if field in demographics:
            segs = get_date_segments(demographics[field])
            if segs: segments_by_date.append(segs)
    # Children and siblings
    for group in ["children_dobs", "siblings_dobs"]:
        for date in demographics.get(group, []):
            segs = get_date_segments(date)
            if segs: segments_by_date.append(segs)
    # Collect all 2-digit/4-digit date parts for mixing
    parts = [seg for field in segments_by_date for seg in field]
    # Add all single-date combos that are 6 digits
    for segs in segments_by_date:
        # Any permutation of 3 parts (from one date): day+month+year2, etc.
        for comb in permutations(segs, 3):
            mpin = ''.join(comb)
            if len(mpin) == 6:
                all_mpin_set.add(mpin)
        # 6-digit full year + day or similar
        if len(segs) == 4: # e.g., DD MM YY YYYY
            # YYYY+DD
            all_mpin_set.add(segs[3] + segs[0])
            all_mpin_set.add(segs[3] + segs[1])
            all_mpin_set.add(segs[0] + segs[3])
            all_mpin_set.add(segs[1] + segs[3])
    # Mix *across* different dates: all tuples of (2-part) from 3 different dates
    if len(segments_by_date) >= 3:
        for date_combo in combinations(segments_by_date, 3):
            for prod in product(date_combo[0], date_combo[1], date_combo[2]):
                mpin = ''.join(prod)
                if len(mpin) == 6:
                    all_mpin_set.add(mpin)
    # Mix *across* any 2-digit part from any source (more aggressive)
    if len(parts) >= 3:
        for prod in combinations(parts, 3):
            prod_str = ''.join(prod)
            if len(prod_str) == 6:
                all_mpin_set.add(prod_str)
    # Phone and pincode substrings
    if is_valid_pincode(demographics.get("pincode", "")):
        all_mpin_set.add(demographics["pincode"])
    if is_valid_phone_number(demographics.get("phone_number", "")):
        phone = demographics["phone_number"]
        all_mpin_set.update(extract_number_mpins(phone, 6))
        all_mpin_set.add(phone)  # rare but conservative
    return all_mpin_set

def check_mpin_strength(phone_number, mpin):
    if not is_valid_mpin(mpin):
        return "INVALID", [], "Invalid MPIN: Must be a 6-digit number."
    if not is_valid_phone_number(phone_number):
        return "INVALID", [], "Invalid phone number: Must be a 10-digit number."
    common_mpins = load_common_mpins_6digit()
    if mpin in common_mpins:
        return "WEAK", ["COMMONLY_USED"], "MPIN is commonly used."
    demographics = load_demographics(phone_number)
    if not demographics:
        print(f"No demographics found for phone number {phone_number}.")
        demographics = get_manual_demographics(phone_number)
        if not demographics:
            return "STRONG", [], "No demographics provided; MPIN not derived from demographics."
    demog_mpins = generate_demographic_mpins(demographics)
    if mpin in demog_mpins:
        return "WEAK", ["DEMOGRAPHIC_MATCH"], "MPIN uses personal or family information (DOB, pincode, etc)."
    return "STRONG", [], "MPIN is strong."

if __name__ == "__main__":
    phone_number = input("Enter phone number (10 digits): ").strip()
    mpin = input("Enter a 6-digit MPIN: ").strip()
    strength, reasons, message = check_mpin_strength(phone_number, mpin)
    print(f"\nPhone Number: {phone_number}")
    print(f"MPIN: {mpin}")
    print(f"Strength: {strength}")
    print(f"Reasons: {reasons}")
    print(f"Message: {message}")



Phone Number: 7901237648
MPIN: 246802
Strength: WEAK
Reasons: ['COMMONLY_USED']
Message: MPIN is commonly used.
